In [1]:
import torch
import torch.nn as nn
from utils import train
from bin_packing_dataset import BinPackingDataset
from bin_packing_model import BinPackingLSTMModel
from torch.utils.data import random_split

In [2]:
# Fijamos la semilla para que los resultados sean reproducibles
SEED = 23

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# Algunas constantes

# definimos el dispositivo que vamos a usar
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS

NUM_WORKERS = 0 # max(os.cpu_count() - 1, 1)  # número de workers para cargar los datos


print(f"Device: {DEVICE}")
print(f"Num Workers: {NUM_WORKERS}")

Device: mps
Num Workers: 0


### Exploración del Dataset

In [4]:
#Creacion del dataset de entrenamiento, validacion y test

full_dataset = BinPackingDataset('data')
print('Full dataset size:', len(full_dataset))
container_tensor, boxes_tensor = full_dataset[0]
print('Container:', container_tensor)
print('Boxes:', boxes_tensor)

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [int(0.7*len(full_dataset)), int(0.20*len(full_dataset)), int(0.10*len(full_dataset))])
print('Train dataset size:', len(train_dataset))
print('Val dataset size:', len(val_dataset))
print('Test dataset size:', len(test_dataset))

Full dataset size: 81000
Container: tensor([14., 11.])
Boxes: tensor([[ 8.,  7.],
        [12.,  2.],
        [ 1.,  2.],
        [ 2.,  7.],
        [ 4.,  8.],
        [ 0.,  0.]])
Train dataset size: 56700
Val dataset size: 16200
Test dataset size: 8100


In [36]:
# Collate para manejar secuencias de diferentes longitudes
import torch.nn.utils.rnn as rnn_utils

def custom_collate_fn_with_padding(batch):
    """
    Collate function que mantiene la estructura de contenedor y agrega padding a las secuencias de cajas.
    
    Args:
        batch (list): Lista de tuplas (contenedor, cajas).
        
    Returns:
        tuple: (contenedores, cajas_padded, longitudes) donde:
            - contenedores: Tensor de tamaño (batch_size, 2).
            - cajas_padded: Tensor de tamaño (batch_size, max_len, 2) con padding.
            - longitudes: Tensor de tamaños originales de las secuencias de cajas.
    """
    containers = torch.stack([item[0] for item in batch])  # Contenedores como tensor
    boxes = [item[1] for item in batch]  # Lista de cajas
    
    # Padding de las secuencias de cajas (rellenar con ceros hasta la longitud máxima en el batch)
    boxes_padded = rnn_utils.pad_sequence(boxes, batch_first=True)
    
    # Longitudes originales de cada secuencia de cajas
    lengths = torch.tensor([len(b) for b in boxes])
    
    return containers, boxes_padded



BATCH_SIZE = 500
mock_loader = torch.utils.data.DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn_with_padding)

x, y = next(iter(mock_loader))
print('Tamaño del primer contenedor:', x.shape)
print('Tamaño de las cajas del primer contenedor:', y.shape)


train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate_fn_with_padding)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=custom_collate_fn_with_padding)

Tamaño del primer contenedor: torch.Size([500, 2])
Tamaño de las cajas del primer contenedor: torch.Size([500, 11, 2])


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AutoRegressiveBinPackingModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, max_dim, n_layers, dropout=0.1):
        super(AutoRegressiveBinPackingModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.max_dim = max_dim
        
        # Embedding para entrada (contenedor y cajas)
        self.embedding = nn.Linear(input_dim, hidden_dim)
        
        # LSTM para modelar secuencias
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout
        )
        
        # Proyección para obtener logits de distribuciones de width y height
        self.fc_width = nn.Linear(hidden_dim, max_dim + 1)  # +1 para incluir el token EOS
        self.fc_height = nn.Linear(hidden_dim, max_dim + 1)
    
    def forward(self, container, target_seq=None, seq_len=100, teacher_forcing_ratio=0.5):
        """
        Args:
            container: Tensor con las dimensiones del contenedor (batch_size, input_dim).
            target_seq: Secuencia objetivo durante entrenamiento (batch_size, seq_len, input_dim).
            seq_len: Longitud máxima de secuencia durante generación.
            teacher_forcing_ratio: Probabilidad de usar teacher forcing (0.0 a 1.0).
        
        Returns:
            Logits para distribuciones de width y height.
        """
        container_emb = self.embedding(container).unsqueeze(1)  # (batch_size, 1, hidden_dim)
        
        # Para almacenar las salidas durante la generación
        outputs_width = []
        outputs_height = []
        
        # Estado inicial
        generated_seq = container_emb
        hidden = None
        
        for t in range(seq_len):
            output, hidden = self.lstm(generated_seq, hidden)  # (batch_size, 1, hidden_dim)
            
            # Logits para width y height
            logits_width = self.fc_width(output[:, -1, :])  # (batch_size, max_dim+1)
            logits_height = self.fc_height(output[:, -1, :])  # (batch_size, max_dim+1)
            outputs_width.append(logits_width)
            outputs_height.append(logits_height)
            
            if target_seq is not None and torch.rand(1).item() < teacher_forcing_ratio:
                # Usar la secuencia objetivo (teacher forcing)
                next_box = target_seq[:, t, :]  # (batch_size, 2)
            else:
                # Sampleo de la predicción
                prob_width = F.softmax(logits_width, dim=-1)  # (batch_size, max_dim+1)
                prob_height = F.softmax(logits_height, dim=-1)  # (batch_size, max_dim+1)
                next_width = torch.multinomial(prob_width, num_samples=1)  # (batch_size, 1)
                next_height = torch.multinomial(prob_height, num_samples=1)  # (batch_size, 1)
                next_box = torch.cat([next_width, next_height], dim=1)  # (batch_size, 2)
            
            # Preparar la entrada para el siguiente paso
            next_box_emb = self.embedding(next_box.float())  # Convertir a embedding
            # Concatenamos la secuencia generada con la nueva caja
            generated_seq = torch.cat([generated_seq, next_box_emb.unsqueeze(1)], dim=1)
        
        # Apilar las salidas
        logits_width = torch.stack(outputs_width, dim=1)  # (batch_size, seq_len, max_dim+1)
        logits_height = torch.stack(outputs_height, dim=1)  # (batch_size, seq_len, max_dim+1)
        return logits_width, logits_height


In [63]:
# Configuración
MAX_DIM = 20
input_dim = 2
hidden_dim = 30
n_layers = 2
dropout = 0
epochs = 50
teacher_forcing_ratio = 1.0  # Iniciamos con teacher forcing completo
teacher_forcing_decay = 0.95  # Decae 5% por época

# Modelo
model = AutoRegressiveBinPackingModel(input_dim, hidden_dim, MAX_DIM, n_layers, dropout)
model.to(DEVICE)
# Hiperparámetros
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
teacher_forcing_ratio = 0.5  # 50% de probabilidad de usar teacher forcing

for epoch in range(epochs):
    model.train()
    teacher_forcing_ratio *= teacher_forcing_decay  # Reducimos el ratio por cada época
    train_loss = 0
    
    for container, target_seq in train_dataloader:
        container = container.to(DEVICE)
        target_seq = target_seq.to(DEVICE)
        
        optimizer.zero_grad()
        logits_width, logits_height = model(
            container, 
            target_seq=target_seq, 
            seq_len=target_seq.size(1), 
            teacher_forcing_ratio=teacher_forcing_ratio
        )
        
        # Aplanar las salidas para calcular la pérdida
        target_width = target_seq[:, :, 0].long()  # (batch_size, seq_len)
        target_height = target_seq[:, :, 1].long()  # (batch_size, seq_len)
        loss_width = criterion(logits_width.view(-1, MAX_DIM+1), target_width.view(-1))
        loss_height = criterion(logits_height.view(-1, MAX_DIM+1), target_height.view(-1))
        
        loss = loss_width + loss_height
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_dataloader)}")




Epoch 1/50, Loss: 5.7722923504678825
Epoch 2/50, Loss: 4.6794328313124804
Epoch 3/50, Loss: 3.9276059991435
Epoch 4/50, Loss: 3.4729884591018942
Epoch 5/50, Loss: 3.2202146722559344
Epoch 6/50, Loss: 3.0963145766341897
Epoch 7/50, Loss: 3.007576448875561
Epoch 8/50, Loss: 2.956689608724494
Epoch 9/50, Loss: 2.9169732917819107
Epoch 10/50, Loss: 2.876465441887839
Epoch 11/50, Loss: 2.851640218182614
Epoch 12/50, Loss: 2.8290247415241443
Epoch 13/50, Loss: 2.817302517723619
Epoch 14/50, Loss: 2.812607915777909
Epoch 15/50, Loss: 2.7937937305684675
Epoch 16/50, Loss: 2.7811428643109504
Epoch 17/50, Loss: 2.7702592841365883
Epoch 18/50, Loss: 2.768784288774457
Epoch 19/50, Loss: 2.7553714513778687
Epoch 20/50, Loss: 2.7502574042270056
Epoch 21/50, Loss: 2.753992929793241
Epoch 22/50, Loss: 2.7502171533149586
Epoch 23/50, Loss: 2.7365178413558424
Epoch 24/50, Loss: 2.737500728222362
Epoch 25/50, Loss: 2.734319074112072
Epoch 26/50, Loss: 2.735385166971307
Epoch 27/50, Loss: 2.73109887148204

In [64]:
from models import BinPackingGame, Box, ResolvedBinPackingGameResult
dataset_keys = set()

def tensor_to_box(tensor):
    return Box(int(tensor[0].item()), int(tensor[1].item()))
    

for container_tensor, boxes_tensor in test_dataset:

    boxes = [tensor_to_box(tensor) for tensor in boxes_tensor]

    game = BinPackingGame(tensor_to_box(container_tensor), boxes)
    game_key = game.generate_unique_key()
    dataset_keys.add(game_key)

In [76]:
def generate_sequence(model, container, max_seq_len=10, teacher_forcing_ratio=0.0):
    """
    Genera una secuencia de cajas para un contenedor dado utilizando un modelo entrenado.
    
    Args:
        model: El modelo entrenado.
        container: Tensor con las dimensiones del contenedor (batch_size, input_dim).
        max_seq_len: Longitud máxima de la secuencia que se desea generar.
        teacher_forcing_ratio: Probabilidad de usar teacher forcing (aunque normalmente se usa 0.0 aquí).
        
    Returns:
        Secuencia generada de dimensiones (seq_len, 2), con el formato de (width, height).
    """
    model.eval()  # Poner el modelo en modo evaluación
    
    # Verifica que el contenedor sea bidimensional
    if container.ndim == 1:
        container = container.unsqueeze(0)  # Convertir a (1, input_dim)
    container = container.to(DEVICE)  # (batch_size, input_dim)
    
    # Embedding inicial del contenedor
    container_emb = model.embedding(container).unsqueeze(1)  # (batch_size, 1, hidden_dim)

    # Inicializa la secuencia generada y el estado oculto del LSTM
    generated_seq = container_emb
    hidden = None
    generated_boxes = []

    with torch.no_grad():
        for _ in range(max_seq_len):
            # Paso del LSTM
            output, hidden = model.lstm(generated_seq, hidden)  # (batch_size, seq_len, hidden_dim)
            
            # Logits para predicciones de width y height
            logits_width = model.fc_width(output[:, -1, :])  # (batch_size, max_dim+1)
            logits_height = model.fc_height(output[:, -1, :])  # (batch_size, max_dim+1)
            
            # Predicciones
            prob_width = F.softmax(logits_width, dim=-1)
            prob_height = F.softmax(logits_height, dim=-1)
            next_width = torch.multinomial(prob_width, num_samples=1).squeeze(-1)  # (batch_size,)
            next_height = torch.multinomial(prob_height, num_samples=1).squeeze(-1)  # (batch_size,)

            # Construir la siguiente caja
            next_box = torch.stack([next_width, next_height], dim=1)  # (batch_size, 2)
            generated_boxes.append(next_box.cpu().numpy())  # Guardar la predicción
            
            # Embedding de la siguiente caja
            next_box_emb = model.embedding(next_box.float().to(DEVICE)).unsqueeze(1)  # (batch_size, 1, hidden_dim)

            # Actualiza la secuencia generada agregando la nueva caja
            generated_seq = torch.cat([generated_seq, next_box_emb], dim=1)

    return generated_boxes


# Ejemplo de uso:
container = torch.tensor([[10, 10]], dtype=torch.float32)  # Ejemplo de contenedor (width=10, height=10)
print(f"{container.shape=}")
generated_seq = generate_sequence(model, container, max_seq_len=10, teacher_forcing_ratio=0.0)

# Mostrar la secuencia generada
print("Secuencia generada:")
for box in generated_seq:
    print(f"Ancho: {box[0][0]}, Alto: {box[0][1]}")


container.shape=torch.Size([1, 2])
Secuencia generada:
Ancho: 3, Alto: 3
Ancho: 9, Alto: 8
Ancho: 11, Alto: 2
Ancho: 7, Alto: 2
Ancho: 8, Alto: 5
Ancho: 0, Alto: 1
Ancho: 0, Alto: 0
Ancho: 7, Alto: 0
Ancho: 0, Alto: 0
Ancho: 0, Alto: 0


In [ ]:
from typing import Counter

from models import BinPackingGame, Box


# Generación de cajas
container_width = 8
container_height = 8

attempts = 100
valid_games = 0
games = []
unique_games = set()
coverages = set()
boxes_count = Counter()
new_games = 0
model.eval()
for i in range(attempts):
    container = torch.tensor([[container_width, container_height]],dtype=torch.float32).to(DEVICE)
    generated_boxes = generate_sequence(model, container, max_seq_len=10, teacher_forcing_ratio=0.0)

    # Paso 2: Convertir a lista de tuplas
    # box_list = [tensor_to_box(tensor) for tensor in output]

    boxes = [Box(int(box_width), int(box_height)) for(box_width, box_height) in generated_boxes]

    valid_boxes = [box for box in boxes if box.width > 0 and box.height > 0]

    game = BinPackingGame(Box(container_width, container_height), valid_boxes)
    games.append(game)
    result = game.solve()
    if isinstance(result, ResolvedBinPackingGameResult):
        valid_games += 1
        game_key = game.generate_unique_key()
        boxes_count[len(game.boxes)] += 1
        # print(f'{game_key=}')
        if game_key not in unique_games:
            unique_games.add(game_key)
            coverages.add(game.coverage())
            if game_key not in dataset_keys:
                new_games += 1

print(f"Valid games: {valid_games}/{attempts}")
print(f"Unique games: {len(unique_games)}/{attempts}")
print(f"unique_games keys: {unique_games}")
print(f"Coverages: {coverages}")
print(f"New games: {new_games}")
print(f"Boxes count: {boxes_count}")
[print(f'{game}') for game in games]


Valid games: 70/100
Unique games: 42/100
unique_games keys: {'c46fd767b93f191d34251bfcc9c31e16', '7360981b7d7de4a07a64af58c3726227', 'bf184ed5419ff413cde64f2fa28f3e7d', '40dfa7ba1f4b2b3d9373780a76f4d033', '6f2984b9e9017d373b4bcd89e50674a4', 'b68746c88a269991906cdc69e82e360e', '2ea69f3fa1270e76ff0b70bd56d946a1', '1943513420048c4b1ebf8b41f330c366', 'a57db73d2d4532b27d6f8dfd87a4e62f', 'd96f163ce5de04e2b98f1a3ef123b3f8', '88d0d9418e7d79024cb201a033a53293', '3bf244e0874c6e2e1215ebb22f59c3b9', '500d8ffc0fd7eb4e1e472448aa92e17c', '3420c1115e18f942883a0bad984ddd9b', 'd3246c7f3bef43bb5fba936b37828f4b', 'd19aedb66799e8563c24046d52980ccb', '779aedcec57697ea3f0829bda5c4b89a', '6d13666e3b6f45cb7f1fcee9100959f9', '4a218ff8e2bcfc984bdaed7fbfbcd04f', 'dc553004cfdfe266f1cb3a029cc59c23', '02c32d31ba63860306b4f4e0ef4c4e18', 'c4ee3ff94e73fe6c6de1991ac596cdc7', '9f8f7c19dc52df6a5e9a0af94f274772', '111a547ef122c65aff6b216eb6b09626', 'ba0d8294824c0987370b3bc20385f036', 'c333852032773fc6f0e13ff6f877524a', 'd9

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]